<a href="https://colab.research.google.com/github/Anshad-Aziz/Machine_Learning/blob/main/Mobile_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import re
import pickle

In [2]:
df=pd.read_csv('mobile_recommendation_system_dataset.csv')

In [3]:
df.head(5)

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [4]:
df.describe()

,ratings
count,2546.000000
mean,4.295797
std,0.214691
min,2.900000
25%,4.200000
50%,4.300000
75%,4.400000
max,5.000000


In [5]:
def random_values(col_name:str,total=6):
  for i in range(0,total):
    index=np.random.randint(df.shape[0])
    print(index,':',df[col_name].iloc[index])

In [6]:
random_values('corpus')

1568 : Storage256  SystemiOS 16Processor TypeA15 Bionic Chip 12MP 12MP 12MP 12MP 63MP 12MP 12MP 5G notMentioned Display Size15.49 cm (6.1 inch)Resolution2532 x 1170 PixelsResolution TypeSuper Retina XDR DisplayGPU5 CoreDisplay TypeSuper Retina XDR DisplayOther Display FeaturesHDR Display, True Tone, Wide Colour (P3), Haptic Touch, Contrast Ratio: 20,00,000:1, Max Brightness: 800 nits, Peak Brightness: 1,200 nits, Fingerprint Resistant Oleophobic Coating, Support for Display of Multiple Languages and Characters Simultaneously
1027 : Storage256 GBRAM12  SystemAndroid 13Processor TypeMediatek Dimensity 7200 5GProcessor Speed2.8 50MP 8MP 2MP 50MP 8MP 2MP 50MP 50MP 5G Capacity4600 Display Size17.22 cm (6.78 Inch)Resolution2400 x 1080 PixelsResolution TypeFull HD+ AMOLED DisplayDisplay TypeFull HD+ AMOLED Display
1990 : Storage128  SystemiOS 15Processor TypeA15 Bionic Chip 12MP 12MP 12MP 12MP 63MP 12MP 12MP 5G notMentioned Display Size15.49 cm (6.1 inch)Resolution2532 x 1170 PixelsResolution

In [7]:
df['corpus']=df['corpus'].str.lower()

In [8]:
random_values('corpus')

570 : storage64 gbram3 gbexpandable storage256gb  systemandroid 11processor typequalcomm snapdragon 439processor speed2 13mp 2mp 2mp 13mp 2mp 2mp 13mp 2mp 2mp 8mp 8mp 4g capacity5000 display size16.54 cm (6.51 inch)resolution1600 x 720 pixelsresolution typehd+display typehd+ lcd (ips) display
1916 : storage128 gbram8  systemandroid 11processor typequalcomm snapdragon 870processor speed3.2 48mp 13mp 13mp 48mp 13mp 13mp 48mp 13mp 13mp 48mp 32mp 32mp 5g capacity4300 display size16.66 cm (6.56 inch)resolution2376 x 1080 pixelsresolution typefull hd+gpuadreno 650display typefull hd+ amoled displayother display features19.8:9 aspect ratio, 92.70% screen-to-body ratio, 120 hz refresh rate, 240 hz response rate, schott xensation up display protection
1387 : storage128 gbram8 gbexpandable storage1tb  systemandroid 13processor typeunisoc tiger t606processor speed1.6 16mp 8mp 4g capacity5000 display size16.76 cm (6.6 inch)resolution1612 x 720resolution typehd+gpuarm mali g57 mp1display typeips lc

In [9]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [10]:
df.dropna(inplace=True)

In [13]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [14]:
df.sample(10)

,name,ratings,price,imgURL,corpus
1165,"realme 9i 5G (Metallica Gold, 64 GB)",4.5,14999,https://rukminim2.flixcart.com/image/312/312/x...,storage64 gbram4 gbexpandable storage1tb syst...
1752,"realme Narzo 50A (Oxygen Green, 128 GB)",4.4,11535,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram4 gbexpandable storage256gb s...
105,"realme 9 5G (Meteor Black, 64 GB)",4.5,13499,https://rukminim2.flixcart.com/image/312/312/l...,storage64 gbram4 gbexpandable storage1tb syst...
1330,"Tecno Spark Go 2021 (Galaxy Blue, 32 GB)",4.2,8288,https://rukminim2.flixcart.com/image/312/312/k...,storage32 gbram2 gbexpandable storage256gb sy...
65,"vivo Y16 (Drizzling Gold, 128 GB)",4.3,12797,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram4 gbexpandable storage1tb sys...
1755,"REDMI Note 10 (Aqua Green, 128 GB)",4.3,17999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
1806,"SAMSUNG Galaxy A71 (Prism Crush Black, 128 GB)",4.2,30537,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram8 gbexpandable storage512gb s...
219,"REDMI Note 8 Pro (Halo White, 128 GB)",4.5,17999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
1627,"OnePlus 7T Pro (Haze Blue, 256 GB)",4.5,32537,https://rukminim2.flixcart.com/image/312/312/k...,storage256 gbram8 systemandroid pie 10process...
1898,"Redmi Note 7S (Sapphire Blue, 32 GB)",4.4,11999,https://rukminim2.flixcart.com/image/312/312/k...,storage32 gbram3 gbexpandable storage256gb sy...


In [15]:
random_values('imgURL')

1391 : https://rukminim2.flixcart.com/image/312/312/k7usyvk0/mobile/z/f/6/mi-redmi-note-9-b0784d862m-original-imafqy27hbtc37gv.jpeg?q=70
529 : https://rukminim2.flixcart.com/image/312/312/mobile/y/u/a/mi-redmi-note-4-na-original-imaeqdxgrdhxgkcx.jpeg?q=70
1228 : https://rukminim2.flixcart.com/image/312/312/kl9rssw0/mobile/u/4/k/e7-power-pamh0013in-pamh0011in-pamh0016in-motorola-original-imagyfu4dkpf6mth.jpeg?q=70
968 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/m/b/9/spark-10-5g-ki8-tecno-original-imagpa6qxtdz4c3b.jpeg?q=70
736 : https://rukminim2.flixcart.com/image/312/312/k0lbdzk0pkrrdj/mobile/z/j/a/mi-redmi-note-7-pro-mzb7465in-original-imafhvrdmeyweauc.jpeg?q=70
1097 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/4/4/2/spark-9t-kg7h-tecno-original-imagh4sgkzryrbbz.jpeg?q=70


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cont_vect=CountVectorizer(max_features=1000)

In [18]:
arrays=cont_vect.fit_transform(df['corpus'])

In [20]:
vectors = arrays.toarray()

In [22]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '07bother', '08', '09',
       '0mp', '0primary', '0processor', '10', '100', '1000', '100000',
       '1000000', '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300',
       '1300processor', '1330', '1334', '135', '13mp', '13processor',
       '14', '1400', '144', '1440', '144hz', '1480', '14processor', '15',
       '1500', '1544', '15processor', '16', '160', '1600', '16000',
       '1600resolution', '1612', '16mp', '16processor', '18', '180',
       '1800', '180hz', '19', '1920', '1920hz', '1processor', '20', '200',
       '2000000', '200mp', '20mp', '21', '22', '23', '2340', '24', '240',
       '2400', '2400resolution', '2408', '240hz', '2460', '24mp', '25m

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
similarity=cosine_similarity(vectors)

In [25]:
similarity.shape

(2534, 2534)

In [27]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, 0.9999999999999997),
 (303, 0.9999999999999997),
 (616, 0.9850746268656713),
 (1461, 0.9850746268656713),
 (457, 0.970149253731343),
 (1027, 0.970149253731343),
 (1735, 0.970149253731343),
 (569, 0.9468131937613656),
 (1123, 0.9468131937613656),
 (1540, 0.9468131937613656)]

In [31]:
def recommend(mobile):
  mobile_index=df[df['name']==mobile].index[0]
  similarity_array=similarity[mobile_index]
  similar_10_mobiles=sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]

  for i in similar_10_mobiles:
    print(df['name'].iloc[i[0]])

In [32]:
recommend('APPLE iPhone 13 mini (Blue, 128 GB)')

APPLE iPhone 13 mini ((PRODUCT)RED, 128 GB)
APPLE iPhone 13 mini (Starlight, 128 GB)
APPLE iPhone 13 mini (Green, 128 GB)
APPLE iPhone 13 mini (Midnight, 128 GB)
APPLE iPhone 13 mini (Green, 256 GB)
APPLE iPhone 13 mini (Midnight, 512 GB)
APPLE iPhone 13 mini (Pink, 512 GB)
APPLE iPhone 13 mini (Starlight, 256 GB)
APPLE iPhone 13 mini (Midnight, 256 GB)
APPLE iPhone 13 mini (Pink, 256 GB)


In [33]:
recommend('OnePlus 7T Pro (Haze Blue, 256 GB)')

OPPO Reno10 Pro+ 5G (Silvery Grey, 256 GB)
OPPO Reno10 Pro 5G (Glossy Purple, 256 GB)
OPPO Reno10 Pro 5G (Silvery Grey, 256 GB)
vivo X90 (Breeze Blue, 256 GB)
vivo X90 (Breeze Blue, 256 GB)
vivo X90 (Asteroid Black, 256 GB)
vivo X60 Pro+ (Emperor Blue, 256 GB)
vivo X90 (Breeze Blue, 256 GB)
vivo V27 Pro 5G (Noble Black, 256 GB)
vivo V27 Pro 5G (Magic Blue, 256 GB)


In [34]:
os.makedirs(r'src\model',exist_ok=True)

In [35]:
pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [36]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))